# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df=pd.read_csv('/datasets/data.csv')
df['education']=df['education'].str.lower()
df.head(15)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Расшифровка столбцов


children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

1) В колонках purpose,education,education_id,	family_status,family_status_id,gender,income_type	debt	 категориальные переменные. В остальных колличественые

2) Корректное название колонок, по этому ни чего менять не нужно

3) Имеются пропуски в данных в следующих колонка total_income - ежемесячный доход,days_employed - общий трудовой стаж в днях и это колличественые переменные

4) в колонке days_employed имеются положительные и отрицательные значения

5) в колонке education категория образования в разных регистрах, по этому привел к общему регистру

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
df.isna().sum()# подсчёт пропусков



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**


Кол-во пропусков в ежемесячном доходе и общем трудовом стаже одинаковые


### Замена типа данных

In [5]:
columns_to_replace=['total_income','days_employed']# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for columns in columns_to_replace:
    df[columns] = df[columns].fillna('NaN')
df.isna().sum()# подсчёт пропусков    
display(df.head(15))
df.info()
print()
print('Заменил тип данных в столбце total_income')
print()
# Так как столбец 'total_income' изменил тип данных object, по этому его нужно перевести в числовой тип данных
df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce') 
df.info()
print()
# Меняю значение кол-во детей с -1 на 1, потому что это ошибка в записи
print('Делаю проверку, заменил кол-во детей с -1 на 1')
df.loc[df.loc[:,'children']==-1,'children']=1
print(df[df['children']==-1].count()) # Вывожу те строчки, где кол-во детей =-1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.42261,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.07787,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97192,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  object
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  object
 11  purpose           21525 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB

Заменил тип данных в столбце total_income

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype

**Вывод** 

1) Все пропуски обработал  

2) После того как обработал пропуски изменился тип данных в столбце total_income

3) В дальнейшем было обнаружено, что имеется 47 строк, в столбце 'children'=-1, уверен что это человечиский фактор и  данное значение нужно заменить на 1



In [6]:
### определяю какой кол-во возростов имеется, для выявления аномалий,


In [7]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [8]:
### Заменяю 0 возрост на пропущенные данные 'NaN'


In [9]:
df.loc[df['dob_years']==0,'dob_years']='NaN'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  object 
 2   dob_years         21525 non-null  object 
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 2.0+ MB


In [10]:
### Произошла замена типов данных в колонке dob_years, по этому нужно поменять тип данных на числовой   и делаю проверку       

In [11]:
df['dob_years'] = pd.to_numeric(df['dob_years'], errors='coerce') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  object 
 2   dob_years         21424 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.0+ MB


### Обработка дубликатов

In [12]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)
df.duplicated().sum()# подсчёт явных дубликатов

0

**Вывод**

От явных дубликатов избавились. При это их было 0,25% от общего кол-ва записей или 54 шт  

### Лемматизация

In [13]:

import pandas as pd
from pymystem3 import Mystem
m = Mystem() 

def lemmatiz_funkci(row):
    lemmas_spisok = m.lemmatize(row['purpose']) #лемматизируем ячейку с целью кредита 
    lemmas=','.join(lemmas_spisok)
    return lemmas


df['lemmas'] = df.apply(lemmatiz_funkci, axis=1) 
display(df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"покупка, ,жилье,\n"
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"приобретение, ,автомобиль,\n"
2,0,-5623.42261,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"покупка, ,жилье,\n"
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"дополнительный, ,образование,\n"
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"сыграть, ,свадьба,\n"
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,"покупка, ,жилье,\n"
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,"операция, ,с, ,жилье,\n"
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,"образование,\n"
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,"на, ,проведение, ,свадьба,\n"
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,"покупка, ,жилье, ,для, ,семья,\n"


Расширю значения в лемотизации

In [38]:
df['purpose_lem'] = df['purpose'].apply(lambda x: m.lemmatize(x))
df['purpose_lem'].value_counts().to_frame()

,purpose_lem
"[автомобиль, \n]",972
"[свадьба, \n]",791
"[на, , проведение, , свадьба, \n]",768
"[сыграть, , свадьба, \n]",765
"[операция, , с, , недвижимость, \n]",675
"[покупка, , коммерческий, , недвижимость, \n]",661
"[операция, , с, , жилье, \n]",652
"[покупка, , жилье, , для, , сдача, \n]",651
"[операция, , с, , коммерческий, , недвижимость, \n]",650
"[покупка, , жилье, \n]",646


**Вывод**

После лимитизации  видно, что цель кредита можно категоризировать и выдели уникальные значения: свадьба, недвижимость, автомобиль, оброзование

### Категоризация данных

In [14]:

df.loc[df.loc[:,'lemmas'].str.contains('автомо'),'lemmas']='автомобиль'
df.loc[df.loc[:,'lemmas'].str.contains('свадь'),'lemmas']='свадьба'
df.loc[df.loc[:,'lemmas'].str.contains('недвиж'),'lemmas']='недвижимость'
df.loc[df.loc[:,'lemmas'].str.contains('жиль'),'lemmas']='недвижимость'
df.loc[df.loc[:,'lemmas'].str.contains('образов'),'lemmas']='оброзование'
df.loc[df.loc[:,'lemmas'].str.contains('оброзов'),'lemmas']='оброзование'
display(df.head(5))




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,-5623.42261,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,оброзование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба


In [15]:
print(df['lemmas'].value_counts())

недвижимость    10811
автомобиль       4306
оброзование      4013
свадьба          2324
Name: lemmas, dtype: int64


**Вывод**

Всего  4 категории причин заимствования, больше всего люди берут кредит на недвижимость, затем на автомобиль

In [16]:
###Категоризацию по уроню дохода произвел ниже, при этом я не понял зачем применять функцию cut
### приминение функции qcut я понля, это нужно для того что бы значение интервалов дохода было разное, при этом кол-во людей
### по этим интервала распределяется одинаково. Только зачем использовать функцию cut в данной задаче, я не понимаю, потому что
### данная фунция делает одинковый интервал между интервалами, а кол-во людей в них будет разное. Именно по этому в данном
### примере применима только функци qcut

## Шаг 3. Отвечу на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
def prochent_funkc(colums):
    returned_credit_children=0# имея детей вернули кредит
    no_returned_credit_children=0# имея детей Невернули кредит
    sum_children=0

    returned_credit_no_children=0# Не имея детей вернули кредит,
    no_returned_credit_no_children=0# Не имея детей Невернули кредит
    sum_no_children=0


    for i in range(df.shape[0]):
    #for i in range(50):    

        if df[colums][i]==1:
            sum_children+=1
            if df['debt'][i]==1:
                returned_credit_children+=1
            else:
                no_returned_credit_children+=1
        if df[colums][i]==0:
            sum_no_children+=1
            if df['debt'][i]==1:
                returned_credit_no_children+=1
            else:
                no_returned_credit_no_children+=1             

    returned_credit_children1=returned_credit_children/sum_children*100
    no_returned_credit_children1=no_returned_credit_children/sum_children*100

    returned_credit_no_children1=returned_credit_no_children/sum_no_children*100
    no_returned_no_credit__no_children1=no_returned_credit_no_children/sum_no_children*100
    returned_credit_children=[returned_credit_children1,no_returned_credit_children1,returned_credit_no_children1,no_returned_no_credit__no_children1]
    
    #return returned_credit_children
    return returned_credit_children1,no_returned_credit_children1,returned_credit_no_children1,no_returned_no_credit__no_children1
    

######################################################
returned_credit_children1,no_returned_credit_children1,returned_credit_no_children1,no_returned_no_credit__no_children1 = prochent_funkc('children')
print("имея детей вернули кредит %.2f" % returned_credit_children1)
print("имея детей Невернули кредит %.2f" % no_returned_credit_children1)

print("Не имея детей вернули кредит %.2f" % returned_credit_no_children1)
print("Не имея детей Невернули кредит %.2f" % no_returned_no_credit__no_children1)


имея детей вернули кредит 9.17
имея детей Невернули кредит 90.83
Не имея детей вернули кредит 7.54
Не имея детей Невернули кредит 92.46


In [19]:

df_example = df.groupby('children')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14091,1063,7.54%,92.46%
1,4855,445,9.17%,90.83%
2,2052,194,9.45%,90.55%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%
20,76,8,10.53%,89.47%


**Вывод**

Между наличием детей и возвратом кредита зависимость есть. Так же наблюдается корреляция между количеством детей и вероятностью возврата кредита. Чаще всего кредиты возвращают люди не имеющие детей, а реже всего: люди с двумя детьми. По клиентам с количеством детей больше трех недостаточно данных для выводов.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
  
df.groupby('family_status')['debt'].value_counts(normalize=True,dropna=False)


family_status          debt
Не женат / не замужем  0       0.902491
                       1       0.097509
в разводе              0       0.928870
                       1       0.071130
вдовец / вдова         0       0.934307
                       1       0.065693
гражданский брак       0       0.906529
                       1       0.093471
женат / замужем        0       0.924548
                       1       0.075452
Name: debt, dtype: float64

In [22]:
family_pivot = df.pivot_table('debt', index='family_status', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
family_pivot = family_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
family_pivot.index.name = 'семейное положение'
family_pivot  

,количество должников,количество заемщиков,доля должников
,,,
семейное положение,,,
Не женат / не замужем,274,2810,9.75%
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%


**Вывод** 
Семейное положение влияет на возврат кредита. У не женатых ,и живущих в гражданском браке шанс возврата кредита ниже. Самые надежные заемщики вдовцы.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:

total_income_labels = ['Низкий', 'Ниже среднего', 'Средний', 'Выше среднего', 'Высокий']

df['income_group_rev'] = pd.qcut(df['total_income'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=total_income_labels)
df['income_group_rev'].value_counts()
#print()
#print(df.info())
df.groupby('income_group_rev')['debt'].value_counts(normalize=True)


income_group_rev  debt
Низкий            0       0.920176
                  1       0.079824
Ниже среднего     0       0.916279
                  1       0.083721
Средний           0       0.913437
                  1       0.086563
Выше среднего     0       0.914987
                  1       0.085013
Высокий           0       0.929199
                  1       0.070801
Name: debt, dtype: float64

In [32]:
def income_sorted(total_income):
    
    if total_income <= 107623:
        return 'до 107623 '
    if total_income <= 156043:
        return 'до 156043'
    if total_income <= 195813:
        return 'до 195813'
    if 195813 < total_income:
        return 'от 195813'
    else:
        print('total_income')

income_group = df.loc[:, ('debt','total_income')]
income_group['income_id'] = income_group['total_income'].apply(income_sorted)


total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income
total_income

In [33]:

income_pivot = income_group.pivot_table('debt', index='income_id', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
income_pivot = income_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
income_pivot.index.name = 'доход'
income_pivot  

,количество должников,количество заемщиков,доля должников
,,,
доход,,,
до 107623,427,5364,7.96%
до 156043,468,5363,8.73%
до 195813,293,3260,8.99%
от 195813,383,5364,7.14%


**Вывод**

На шанс возврата кредита влияет уровень дохода. Самый высокий шанс вернуть кредит у клиентов с доходом свыше 195 813.

### Как разные цели кредита влияют на его возврат в срок?

In [34]:
print(df.groupby('lemmas')['debt'].value_counts(normalize=True)) 




lemmas        debt
автомобиль    0       0.906410
              1       0.093590
недвижимость  0       0.927666
              1       0.072334
оброзование   0       0.907800
              1       0.092200
свадьба       0       0.919966
              1       0.080034
Name: debt, dtype: float64


In [40]:
def purpose_sorted(purpose_lem):
    if 'автомобиль' in purpose_lem:
        return 'автомобиль'
    if 'жилье' in purpose_lem or 'недвижимость' in purpose_lem:
        return 'недвижимость'
    if 'свадьба' in purpose_lem:
        return 'свадьба'
    if 'образование':
        return 'образование'
    else:
        print(purpose_lem)

purpose_group = df.loc[:, ('debt','purpose_lem')]
purpose_group['purpose_id'] = purpose_group['purpose_lem'].apply(purpose_sorted)
purpose_and_debt = purpose_group.groupby('purpose_id')['debt'].agg(['sum', 'count'])
purpose_and_debt['proportion'] = purpose_and_debt['sum'] / purpose_and_debt['count']
purpose_and_debt

,sum,count,proportion
purpose_id,,,
автомобиль,403,4306,0.093590
недвижимость,782,10811,0.072334
образование,370,4013,0.092200
свадьба,186,2324,0.080034


In [41]:
df.groupby('lemmas')['debt'].agg('mean')

lemmas
автомобиль      0.093590
недвижимость    0.072334
оброзование     0.092200
свадьба         0.080034
Name: debt, dtype: float64

**Вывод**

Цели кредита также влияют на шанс возврата кредита. Так больше вероятность вернуть кредит у заемщиков берущих кредит на недвижимость, а те кто брал деньги на автомобиль или образование, мене склонны возвращать кредит

## Шаг 4. Общий вывод

На шанс вернуть кредит влияет семейное положение и количество детей:

Так самыми надежными заемщиками являются те кто вообще не имеет детей, они не возвращают кредит всего в 7.54%, в отличие от клиентов имеющих двое детей, которые не так добросовестно относятся к задолженности и не возвращают в 9.49% случаев.
Что касается семейного положения то клиенты не состоящие в браке, не платят по кредитам в 9.75% случаев. А вот вдовцы самая надежная категория, кредиторов 6.57% уклоняются от выплат. Так же есть зависимость между возвратом кредита, и целями, и доходом заемщика:
По целям самые добросовестные заемщики те кто берет деньги на недвижимость, они не выплачивают в 7.23% случаев. Заёмщики берущие деньги на автомобиль не платят в 9.36% случаев.
Клиенты с доходом в диапазоне от 156044 до 195813 не платят по кредиту 8.89% случаях. А те у кого доход свыше 195813 не выплачивают в 7.04%. Надежным заемщиком является, вдовец без детей, взявший кредит на недвижимость и с доходом более 195813.

Рекомендую проанализировать зависимость с возрастом возможно он в большей степени влияет на шанс возврата денег и сказывается на остальной статистике, так я могу предположить что вдовцы и люди с большим количеством детей, чаще будут в более высокой возрастной категории, чем холостые. Так же я не смог придумать как с высокой достоверностью восстановить данные из колонки трудового стажа. Хотелось бы более подробно узнать как собирались эти данные, возможно ошибка конвертации.